In [1]:
import QuantLib as ql
import numpy as np
from timeit import default_timer as timer

import numpy as np
from timeit import default_timer as timer

class Option:
    def __init__(self, calculation_date, maturity, stock_price, strike_price, volatility, dividend_rate, risk_free_rate, option_type):
        self.maturity = maturity
        self.stock_price = stock_price
        self.strike_price = strike_price
        self.volatility = volatility
        self.dividend_rate = dividend_rate
        self.risk_free_rate = risk_free_rate
        self.option_type = option_type
        self.calculation_date = calculation_date
        self.bs_price = -1
        self.mc_price = -1

    def BSM_price(self):
        day_count = ql.Actual365Fixed()
        calendar = ql.UnitedStates()
        ql.Settings.instance().evaluationDate = self.calculation_date
        
        payoff = ql.PlainVanillaPayoff(self.option_type, self.strike_price)
        exercise = ql.EuropeanExercise(self.maturity)
        european_option = ql.VanillaOption(payoff, exercise)
        spot_handle = ql.QuoteHandle(ql.SimpleQuote(self.stock_price))
        flat_ts = ql.YieldTermStructureHandle(ql.FlatForward(self.calculation_date, self.risk_free_rate, day_count))
        dividend_yield = ql.YieldTermStructureHandle(ql.FlatForward(self.calculation_date, self.dividend_rate, day_count))
        flat_vol_ts = ql.BlackVolTermStructureHandle(ql.BlackConstantVol(self.calculation_date, calendar, self.volatility, day_count))
        bsm_process = ql.BlackScholesMertonProcess(spot_handle, 
                                                   dividend_yield, 
                                                   flat_ts, 
                                                   flat_vol_ts)
        european_option.setPricingEngine(ql.AnalyticEuropeanEngine(bsm_process))
        bs_price = european_option.NPV()
        self.bs_price = bs_price
        return self.bs_price
    
    def BSM_Greeks(self):
        day_count = ql.Actual365Fixed()
        calendar = ql.UnitedStates()
        ql.Settings.instance().evaluationDate = self.calculation_date
        
        payoff = ql.PlainVanillaPayoff(self.option_type, self.strike_price)
        exercise = ql.EuropeanExercise(self.maturity)
        european_option = ql.VanillaOption(payoff, exercise)
        spot_handle = ql.QuoteHandle(ql.SimpleQuote(self.stock_price))
        flat_ts = ql.YieldTermStructureHandle(ql.FlatForward(self.calculation_date, self.risk_free_rate, day_count))
        dividend_yield = ql.YieldTermStructureHandle(ql.FlatForward(self.calculation_date, self.dividend_rate, day_count))
        flat_vol_ts = ql.BlackVolTermStructureHandle(ql.BlackConstantVol(self.calculation_date, calendar, self.volatility, day_count))
        bsm_process = ql.BlackScholesMertonProcess(spot_handle, 
                                                   dividend_yield, 
                                                   flat_ts, 
                                                   flat_vol_ts)
        european_option.setPricingEngine(ql.AnalyticEuropeanEngine(bsm_process))
        time_bs_start = timer()
        bs_price = european_option.NPV()
        delta = european_option.delta()
        gamma = european_option.gamma() #second order
        vega = european_option.vega()
        rho = european_option.rho()
        theta = european_option.theta()
        time_bs_end = timer()
        time_bs = time_bs_end - time_bs_start
#         print("Delta is ", delta)#Change in underlying price
#         print("Gamma is ", gamma)#second order: rate of change in price
#         print("Tho is ", rho)#Change in interest rate
#         print("Vega is ", vega)#Change in volatility
#         print("Theta is ", theta)#Change in time to expiration
#         print("Time used for BS calculation in total is ", time_bs, ' seconds')
        greeks_output = [('bs', delta, gamma, rho, vega, theta, time_bs)]
        dataframe = pd.DataFrame(greeks_output)
        dataframe.columns = ['method','delta', 'gamma', 'rho', 'vega', 'theta', 'time']
        return dataframe

    '''
    MC by hand.
    https://qsctech-sange.github.io/Options-Calculator.html#%E5%AE%8C%E6%95%B4%E4%BB%A3%E7%A0%81
    
    '''
    def MC_price(self):
        iteration = 100000
        kind = 1 #call = 1, put = -1
        maturity_in_year = (self.maturity - self.calculation_date)/365
        zt = np.random.normal(0, 1, iteration)

        st = self.stock_price * np.exp((self.risk_free_rate - self.dividend_rate - .5 * self.volatility ** 2) * maturity_in_year + self.volatility * maturity_in_year ** .5 * zt)
        st = np.maximum(kind * (st - self.strike_price), 0)
        self.mc_price = np.average(st) * np.exp(-self.risk_free_rate * maturity_in_year)
        return self.mc_price
    
    '''
    https://chixiaoxue.github.io/2018/08/15/Python%E5%AE%9E%E7%8E%B0%E8%92%99%E7%89%B9%E5%8D%A1%E6%B4%9B%E6%A8%A1%E6%8B%9F%E7%9A%84%E6%9C%9F%E6%9D%83%E4%BC%B0%E5%80%BC/
    '''
    def MC_A_price(self):
        iteration = 1000000
        kind = 1 #call = 1, put = -1
        maturity_in_year = (self.maturity - self.calculation_date)/365
        #zt = np.random.normal(0, 1, iteration)
        np.random.seed(1000)
        ran = np.random.standard_normal((1,1, round(iteration/2)))
        ran = np.concatenate((ran,-ran),axis=2)
        ran = ran-np.mean(ran)
        ran = ran/np.std(ran)
        
        st = self.stock_price * np.exp((self.risk_free_rate - self.dividend_rate - .5 * self.volatility ** 2) * maturity_in_year + self.volatility * maturity_in_year ** .5 * ran)
        st = np.maximum(kind * (st - self.strike_price), 0)
        self.mc_price = np.average(st) * np.exp(-self.risk_free_rate * maturity_in_year)
        return self.mc_price
    
    def MC_A_Greeks(self):
        if self.mc_price == -1:
            self.MC_A_price()
            
        p_original = self.mc_price
        time_bs_start = timer()
        
        original_stock = self.stock_price
        change_stock_price = 0.1
        self.stock_price = original_stock + change_stock_price
        p_plus = self.MC_A_price()
        self.stock_price = original_stock - change_stock_price
        p_minus = self.MC_A_price()
        self.stock_price = original_stock
        mc_delta = (p_plus - p_minus) / (2*change_stock_price)
        mc_gamma = (p_plus - 2*p_original + p_minus) /(change_stock_price*change_stock_price)

        change_r = 0.001
        original_r = self.risk_free_rate
        self.risk_free_rate = original_r + change_r
        p_plus = self.MC_A_price()
        self.risk_free_rate = original_r
        mc_rho = (p_plus - p_original) / change_r
        
        change_sigma = 0.001
        original_sigma = self.volatility
        self.volatility = original_sigma + change_sigma
        p_plus = self.MC_A_price()
        self.volatility = original_sigma
        mc_vega = (p_plus - p_original) / change_sigma
        
        self.calculation_date += 1
        p_plus = self.MC_A_price()
        change_in_time = 1.0 / 365
        mc_theta = (p_plus - p_original) / change_in_time
        
        self.mc_price = p_original
        
        time_bs_end = timer()
        time_mc = time_bs_end - time_bs_start

        greeks_output = [('mc', mc_delta, mc_gamma, mc_rho, mc_vega, mc_theta, time_mc)]
        dataframe = pd.DataFrame(greeks_output)
        dataframe.columns = ['method','delta', 'gamma', 'rho', 'vega', 'theta', 'time']
        return dataframe
    
    def data_set(self):
        '''
        Funtion to return a set of required data for one sample for training purpose.
        
        '''
        if self.bs_price == -1:
            self.BSM_price()
        maturity_in_year = (self.maturity - self.calculation_date)/365
        data_set = (self.stock_price, self.strike_price, maturity_in_year, self.dividend_rate, self.volatility, self.risk_free_rate, self.bs_price)
        return data_set


In [2]:
import datetime
import random
import pandas as pd

'''Date helper functions'''
def xldate_to_datetime(xldate):
    temp = datetime.datetime(1899, 12, 30)
    delta = datetime.timedelta(days=xldate)
    return temp+delta

def ql_to_datetime(d):
    return datetime.datetime(d.year(), d.month(), d.dayOfMonth())

def datetime_to_xldate(date):
    temp = datetime.datetime(1899, 12, 30)
    return (date - temp).days

def random_options(numbers = 0):
    options = []
    start_maturity = datetime.datetime(2020,11,1)
    end_maturity = datetime.datetime(2023,10,30)
    calculation_date = datetime.datetime(2020,10,30)
    
    xldate1 = datetime_to_xldate(start_maturity)
    xldate2 = datetime_to_xldate(end_maturity)
    calculation_xldate = datetime_to_xldate(calculation_date)
    calculation_date = ql.Date(calculation_xldate)
    for number in range(numbers):
        maturity = ql.Date(random.randint(xldate1, xldate2+1))
        stock_price = random.randint(100, 501)
        strike_price = random.randint(7, 651)
        volatility = random.uniform(0.05, 0.90)
        dividend_rate = random.uniform(0, 0.003)
        risk_free_rate = random.uniform(0.001, 0.003)
        option_type = ql.Option.Call
        option = Option(calculation_date, maturity, stock_price, strike_price, volatility, dividend_rate, risk_free_rate, option_type)
        options.append(option)
    return options

def random_options_pd(numbers = 0):
    options = []
    start_maturity = datetime.datetime(2020,11,1)
    end_maturity = datetime.datetime(2023,10,30)
    calculation_date = datetime.datetime(2020,10,30)
    
    xldate1 = datetime_to_xldate(start_maturity)
    xldate2 = datetime_to_xldate(end_maturity)
    calculation_xldate = datetime_to_xldate(calculation_date)
    calculation_date = ql.Date(calculation_xldate)
    
    for number in range(numbers):
        maturity = ql.Date(random.randint(xldate1, xldate2+1))
        stock_price = random.randint(100, 501)
        strike_price = random.randint(7, 651)
        volatility = random.uniform(0.05, 0.90)
        dividend_rate = random.uniform(0, 0.003)
        risk_free_rate = random.uniform(0.001, 0.003)
        option_type = ql.Option.Call
        option = Option(calculation_date, maturity, stock_price, strike_price, volatility, dividend_rate, risk_free_rate, option_type)
        options.append(option.data_set())  
    dataframe = pd.DataFrame(options)
    dataframe.columns = ['stock_price', 'strike_price', 'maturity', 'devidends', 'volatility', 'risk_free_rate', 'call_price']
    return dataframe

def random_options_uniform(numbers = 0, index = 100):
    options = []
    start_maturity = datetime.datetime(2020,11,1)
    end_maturity = datetime.datetime(2023,10,30)
    calculation_date = datetime.datetime(2020,10,30)
    
    xldate1 = datetime_to_xldate(start_maturity)
    xldate2 = datetime_to_xldate(end_maturity)
    calculation_xldate = datetime_to_xldate(calculation_date)
    calculation_date = ql.Date(calculation_xldate)
    
    numbers = 10 
    maturity = np.linspace(xldate1, xldate2+1, 10)
    stock_price = np.linspace(100, 500, 10)
    strike_price = np.linspace(7, 650, 10)
    volatility = np.linspace(0.05, 0.90, 10)
    dividend_rate = np.linspace(0, 0.003, 10)
    risk_free_rate = np.linspace(0.001, 0.003, 10)
    option_type = ql.Option.Call
    
    counter = 0
    for i in range(numbers):
        for j in range(numbers):
            for k in range(numbers):
                for l in range(numbers):
                    for m in range(numbers):
                        for n in range(numbers):
                            counter += 1
                            if counter == index:
                                option = Option(calculation_date, ql.Date(int(maturity[i])), stock_price[j], strike_price[k], volatility[l], dividend_rate[m], risk_free_rate[n], option_type)
                                options.append(option)
    
    return options

def random_options_pd_uniform(numbers = 0):
    options = []
    start_maturity = datetime.datetime(2020,11,1)
    end_maturity = datetime.datetime(2023,10,30)
    calculation_date = datetime.datetime(2020,10,30)
    
    xldate1 = datetime_to_xldate(start_maturity)
    xldate2 = datetime_to_xldate(end_maturity)
    calculation_xldate = datetime_to_xldate(calculation_date)
    calculation_date = ql.Date(calculation_xldate)
    
    numbers = 10 
    maturity = np.linspace(xldate1, xldate2+1, 10)
    stock_price = np.linspace(100, 500, 10)
    strike_price = np.linspace(7, 650, 10)
    volatility = np.linspace(0.05, 0.90, 10)
    dividend_rate = np.linspace(0, 0.003, 10)
    risk_free_rate = np.linspace(0.001, 0.003, 10)
    option_type = ql.Option.Call
    
    for i in range(numbers):
        for j in range(numbers):
            for k in range(numbers):
                for l in range(numbers):
                    for m in range(numbers):
                        for n in range(numbers):
                            option = Option(calculation_date, ql.Date(int(maturity[i])), stock_price[j], strike_price[k], volatility[l], dividend_rate[m], risk_free_rate[n], option_type)
                            options.append(option.data_set())  
    dataframe = pd.DataFrame(options)
    dataframe.columns = ['stock_price', 'strike_price', 'maturity', 'devidends', 'volatility', 'risk_free_rate', 'call_price']
    return dataframe

In [3]:
def DL_prediction_scaled(scaler, model, df):
    scaled_training = scaler.transform(df)
    input_df = pd.DataFrame(scaled_training,columns=df.columns.values)
    greek_input = input_df[['stock_price','strike_price', 'maturity', 'devidends', 'volatility', 'risk_free_rate']].values
    nn_original_price = (model.predict(greek_input)[0][0]  - scaler.min_[6])/scaler.scale_[6] 
    return nn_original_price

def DL_Greeks(scaler, model, df):
    
    time_dl_start = timer()
    
#The Greeks for DL model
#why it's forever negative, problem is my saving variables for Python!!

    change_stock_price = 0.1
    original_price = DL_prediction_scaled(scaler, model, df)
    
    df['stock_price'] = df['stock_price'] + change_stock_price
    p_plus = DL_prediction_scaled(scaler, model, df)

    df['stock_price'] = df['stock_price'] - 2*change_stock_price
    p_minus = DL_prediction_scaled(scaler, model, df)
    df['stock_price'] = df['stock_price'] + change_stock_price

    dl_delta = (p_plus - p_minus) / (2*change_stock_price)
    dl_gamma = (p_plus - 2*original_price + p_minus) /(change_stock_price*change_stock_price)

    change_r = 0.001
    df['risk_free_rate'] = df['risk_free_rate'] + change_r
    p_plus = DL_prediction_scaled(scaler, model, df)
    df['risk_free_rate'] = df['risk_free_rate'] - change_r
    dl_rho = (p_plus - original_price) / change_r

    change_sigma = 0.001
    df['volatility'] = df['volatility'] + change_sigma
    p_plus = DL_prediction_scaled(scaler, model, df)
    df['volatility'] = df['volatility'] - change_sigma
    dl_vega = (p_plus - original_price) / change_sigma

    change_in_time = 1.0 / 365
    df['maturity'] = df['maturity'] - change_in_time
    p_later = DL_prediction_scaled(scaler, model, df)
    dl_theta = (p_later - original_price) / change_in_time

    time_dl_end = timer()
    time_dl = time_dl_end - time_dl_start

    greeks_output = [('dl', dl_delta, dl_gamma, dl_rho, dl_vega, dl_theta, time_dl)]
    dataframe = pd.DataFrame(greeks_output)
    dataframe.columns = ['method','delta', 'gamma', 'rho', 'vega', 'theta', 'time']
    return dataframe

In [4]:
from keras.models import model_from_json

# load json and create model
json_file = open('model_uniform.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
price_model = model_from_json(loaded_model_json)
price_model.load_weights("model_weights_uniform.h5")
print("Loaded model from disk")
price_model.summary()

# evaluate loaded model on test data
price_model.compile(loss='mean_squared_error', optimizer='rmsprop', metrics=['mse'])

Using TensorFlow backend.


Loaded model from disk
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 50)                350       
_________________________________________________________________
dense_2 (Dense)              (None, 100)               5100      
_________________________________________________________________
dense_3 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_4 (Dense)              (None, 50)                5050      
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 51        
Total params: 20,651
Trainable params: 20,651
Non-trainable params: 0
_________________________________________________________________


In [5]:
from keras.models import model_from_json

# load json and create model
json_file = open('model_multi.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
greeks_model = model_from_json(loaded_model_json)
# load weights into new model
greeks_model.load_weights("model_weights_multi.h5")
print("Loaded model from disk")
greeks_model.summary()

# evaluate loaded model on test data
greeks_model.compile(loss='mean_squared_error', optimizer='rmsprop', metrics=['mse'])

Loaded model from disk
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 6)            0                                            
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 50)           350         input_1[0][0]                    
__________________________________________________________________________________________________
dense_2 (Dense)                 (None, 100)          5100        dense_1[0][0]                    
__________________________________________________________________________________________________
dense_3 (Dense)                 (None, 100)          10100       dense_2[0][0]                    
_____________________________________________________________________

In [6]:
from sklearn.preprocessing import MinMaxScaler

train = pd.read_pickle('delta_uniform_train.pkl')
test = pd.read_pickle('delta_uniform_test.pkl')

scaler = MinMaxScaler(feature_range=(0,1))
scaled_training = scaler.fit_transform(train)
scaled_testing = scaler.transform(test)
scaled_training_df = pd.DataFrame(scaled_training,columns=train.columns.values)
scaled_testing_df = pd.DataFrame(scaled_testing,columns=test.columns.values)

print(scaled_training_df.info())
X_train = scaled_training_df[['stock_price','strike_price', 'maturity', 'devidends', 'volatility', 'risk_free_rate']].values
y_train = scaled_training_df[['call_price','delta', 'gamma', 'rho', 'vega', 'theta']].values
X_test = scaled_testing_df[['stock_price', 'strike_price','maturity', 'devidends', 'volatility', 'risk_free_rate']].values
y_test = scaled_testing_df[['call_price','delta', 'gamma', 'rho', 'vega', 'theta']].values

y_train_price = scaled_training_df['call_price'].values
y_train_delta = scaled_training_df['delta'].values
y_train_gamma = scaled_training_df['gamma'].values
y_train_rho = scaled_training_df['rho'].values
y_train_vega = scaled_training_df['vega'].values
y_train_theta = scaled_training_df['theta'].values

y_test_price = scaled_testing_df['call_price'].values
y_test_delta = scaled_testing_df['delta'].values
y_test_gamma = scaled_testing_df['gamma'].values
y_test_rho = scaled_testing_df['rho'].values
y_test_vega = scaled_testing_df['vega'].values
y_test_theta = scaled_testing_df['theta'].values

print("X:", X_train.shape, "Y:",y_train.shape)
 
in_dim = X_train.shape[1]
out_dim = y_train.shape[1]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800000 entries, 0 to 799999
Data columns (total 12 columns):
stock_price       800000 non-null float64
strike_price      800000 non-null float64
maturity          800000 non-null float64
devidends         800000 non-null float64
volatility        800000 non-null float64
risk_free_rate    800000 non-null float64
call_price        800000 non-null float64
delta             800000 non-null float64
gamma             800000 non-null float64
vega              800000 non-null float64
rho               800000 non-null float64
theta             800000 non-null float64
dtypes: float64(12)
memory usage: 73.2 MB
None
X: (800000, 6) Y: (800000, 6)


In [14]:
option = random_options(1)
greeks = option[0].BSM_Greeks().values.tolist()[0]
greeks = tuple(greeks[1:-1])
dataframe = pd.DataFrame([option[0].data_set() + greeks])
dataframe.columns = ['stock_price', 'strike_price', 'maturity', 'devidends', 'volatility', 'risk_free_rate', 'call_price','delta', 'gamma', 'rho', 'vega', 'theta']
bs_greeks = option[0].BSM_Greeks()
mc_greeks = option[0].MC_A_Greeks()
dl_greeks = DL_Greeks(scaler, price_model, dataframe)
bs_greeks['call_price'] = option[0].BSM_price()
mc_greeks['call_price'] = option[0].MC_A_price()
dl_greeks['call_price'] = DL_prediction_scaled(scaler, price_model, dataframe)

input_training = scaler.transform(dataframe)
#print(input_training[0][6:])
input_df = pd.DataFrame(input_training,columns=dataframe.columns.values)
greek_input = input_df[['stock_price','strike_price', 'maturity', 'devidends', 'volatility', 'risk_free_rate']].values
time_greeks_dl_start = timer()
greek_predictions = greeks_model.predict(greek_input)
time_greeks_dl_end = timer()
#print(scaler.min_)
#print(scaler.scale_)
temp1 = greek_predictions[3]
temp2 = greek_predictions[4]
greek_predictions[4] = temp1
greek_predictions[3] = temp2
#original_prediction =[0, 0 ,0, 0, 0, 0]
for i in range(len(greek_predictions)):
    #original_prediction[i] = (greek_predictions[i][0])[0]
    greek_predictions[i] = ((greek_predictions[i][0] - scaler.min_[6+i])/scaler.scale_[6+i])[0]
greek_predictions = [tuple(greek_predictions)]
#original_prediction = [tuple(original_prediction)]
greeks_results = pd.DataFrame(greek_predictions,columns=['call_price', 'delta', 'gamma', 'vega','rho', 'theta'])
greeks_results['method'] = 'multi-dl'
greeks_results['time'] = time_greeks_dl_end - time_greeks_dl_start

df_greeks = pd.concat([bs_greeks, mc_greeks, dl_greeks,greeks_results], sort=False).reset_index(drop=True)
df_greeks['price_diff'] = df_greeks['call_price']/df_greeks['call_price'].iat[0] - 1
df_greeks['delta_diff'] = df_greeks['delta']/df_greeks['delta'].iat[0] - 1
df_greeks['gamma_diff'] = df_greeks['gamma']/df_greeks['gamma'].iat[0] - 1
df_greeks['rho_diff'] = df_greeks['rho']/df_greeks['rho'].iat[0] - 1
df_greeks['vega_diff'] = df_greeks['vega']/df_greeks['vega'].iat[0] - 1
df_greeks['theta_diff'] = df_greeks['theta']/df_greeks['theta'].iat[0] - 1

df_greeks.round(4)


,method,delta,gamma,rho,vega,theta,time,call_price,price_diff,delta_diff,gamma_diff,rho_diff,vega_diff,theta_diff
0,bs,0.6063,0.001,233.8274,212.1624,-34.0508,0.0000,119.7821,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000
1,mc,0.6064,0.001,233.7003,212.7956,-34.1696,0.2931,119.8970,0.0010,0.0000,0.0027,-0.0005,0.0030,0.0035
2,dl,0.6327,0.000,234.3314,216.1962,-30.5594,0.0160,120.0702,0.0024,0.0436,-1.0000,0.0022,0.0190,-0.1025
3,multi-dl,0.6034,0.001,236.1006,212.4272,-36.6623,0.0019,119.4070,-0.0031,-0.0048,-0.0179,0.0097,0.0012,0.0767
